# DSL — Colab finetune (PEFT + W&B)

This notebook clones the repo, installs dependencies (including optional PEFT/LoRA), mounts Drive (optional), and launches a PEFT finetune that logs to Weights & Biases.


Set Runtime → Change runtime type → GPU before running. Use Colab Pro if available.

## 1) Clone repo and install dependencies

The cell below clones the repository and installs required packages. If `bitsandbytes` fails to install on your runtime, you can remove that line and continue without it.

In [ ]:
# Clone the repo and install deps
!git clone https://github.com/sneefyyy/DSL.git /content/DSL || (cd /content/DSL && git pull)
%cd /content/DSL
# Upgrade pip and install base requirements
!python -m pip install --upgrade pip
!pip install -r requirements.txt
# Optional but recommended for PEFT/LoRA and faster training
!pip install -U accelerate peft
# Optional: bitsandbytes for 8-bit optimizations (may require CUDA drivers). If it errors, re-run without this line.
!pip install bitsandbytes || true
# Install wandb for logging
!pip install wandb

## 2) (optional) Mount Google Drive

If you want to read/write checkpoints to Drive, mount it here. Otherwise checkpoints will be written to the Colab VM (ephemeral).

In [ ]:
# Optional: mount Google Drive
mount_drive = False  # set True to mount if you want Drive persistence
output_dir = '/content/dsl-finetuned'
if mount_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    output_dir = '/content/drive/MyDrive/dsl-finetuned'
print('output_dir =', output_dir)

## 3) Provide Hugging Face and W&B credentials (secure)

Enter tokens below. These are stored in environment variables for the notebook session only (not saved to Drive).

In [ ]:
import os
from getpass import getpass
hf_token = getpass('Hugging Face token (or Enter to skip): ')
if hf_token:
    os.environ['HF_TOKEN'] = hf_token
    # optional: login via huggingface-cli if available
    try:
        get_ipython().system_raw(f"huggingface-cli login --token {hf_token}")
    except Exception:
        pass

wandb_key = getpass('Weights & Biases API key (or Enter to skip): ')
if wandb_key:
    os.environ['WANDB_API_KEY'] = wandb_key
    os.environ.setdefault('WANDB_MODE', 'online')
print('HF_TOKEN set:', 'HF_TOKEN' in os.environ, 'WANDB set:', 'WANDB_API_KEY' in os.environ)

## 4) Quick checks: Python, GPU, and installed packages

Confirm GPU visibility and important packages are installed.

In [ ]:
import torch, sys, os
print('Python', sys.version)
print('Torch', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
try:
    import peft
    print('peft OK')
except Exception:
    print('peft not installed or import failed')
import wandb
print('wandb mode =', os.environ.get('WANDB_MODE'))

## 5) Run the finetune (PEFT + W&B)

Adjust model, dataset, and training args as needed. If you saved dataset locally to Drive, use that path for `--dataset_id`. The example below launches the script in the background; remove the ampersand and `&` to run in foreground and stream logs to the notebook output.

In [ ]:
# Configure training command and run
model_name = 'gpt2'
dataset_id = 'middles/dsl-arc-dataset-v0.0.1'  # replace with local path if needed
output_dir = '/content/dsl-finetuned'
use_peft = True
use_wandb = True
wandb_project = 'dsl-finetune'
peft_r = 8
peft_alpha = 32
# Optionally run W&B offline instead of online:
# os.environ['WANDB_MODE'] = 'offline'
cmd = 'python3 src/dsl/finetune_hf.py '
cmd += f"--model_name_or_path {model_name} "
cmd += f"--dataset_id {dataset_id} "
cmd += f"--output_dir {output_dir} "
cmd += '--num_train_epochs 3 --per_device_train_batch_size 4 --per_device_eval_batch_size 4 '
if use_wandb:
    cmd += '--use_wandb '
    cmd += f'--wandb_project {wandb_project} '
if use_peft:
    cmd += '--use_peft '
    cmd += f'--peft_r {peft_r} --peft_alpha {peft_alpha} '
print('Running finetune command:')
print(cmd)
# Start in background so the notebook remains responsive; remove '&' to run in foreground
get_ipython().system_raw(cmd + ' &')
print('Finetune started in background. Use `!ps aux | grep finetune_hf.py` to check, or inspect logs under the output dir.')

## 6) View W&B runs and tables

- Online: open the W&B project page at https://wandb.ai/<your-entity>/<project>
- Offline: runs are stored locally under `./wandb/` and can be inspected in the notebook.

In [ ]:
# Show local wandb offline directories (if any)
import os, glob, json
for d in sorted(glob.glob('wandb/offline-run-*')):
    print(d)
    for f in os.listdir(d)[:50]:
        print('  ', f)
# Search for functional_predictions artifacts in offline runs
for d in glob.glob('wandb/offline-run-*'):
    for p in glob.glob(d + '/**/functional_predictions*', recursive=True):
        print('Found table file:', p)
        try:
            with open(p, 'r') as fh:
                for i, line in enumerate(fh):
                    if i >= 5:
                        break
                    print('  ', line.strip())
        except Exception:
            pass

### Notes and tips
- Replace `middles/dsl-arc-dataset-v0.0.1` with a local dataset path if you uploaded it to Drive.
- Use smaller batch sizes on Colab if you run out of GPU memory.
- To avoid W&B uploads during debugging, set `os.environ['WANDB_MODE']='offline'` before launching the training command.
- If you want the training logs streamed live in the notebook, remove the ampersand `&` at the end of the command invocation in the training cell.